In [22]:
import pandas as pd

# load the harvard database
df = pd.read_csv('../havard_gis/tgaz_chgis_2016-07-06.csv')

df_pre_1750 = df
# choose the level of details, not equal to "路" and "政权" and "国"
df_pre_1750 = df_pre_1750.loc[df_pre_1750["TYPE_SIM"]!="路"]
df_pre_1750 = df_pre_1750.loc[df_pre_1750["TYPE_SIM"]!="政权"]
df_pre_1750 = df_pre_1750.loc[df_pre_1750["TYPE_SIM"]!="国"]
# choose the time period
df_pre_1750 = df_pre_1750.loc[df_pre_1750["BEG"]>1400]
df_pre_1750 = df_pre_1750.loc[df_pre_1750["END"]<=1905]

df_total = df_pre_1750.loc[df_pre_1750["TYPE_SIM"]!="路"]
all_the_places = df_total["NAME_SIM"].unique()

In [23]:
f = open('../novel/ru_lin_wai_shi.txt', encoding="utf8")
text = f.read()
chapters = text.split("　　------------------")

# looop it through the text and remove redundent strings
new_chapers = []
for single_text in chapters:
    single_text = single_text.replace("\n\u3000\u3000一鸣扫描，雪儿校对\n\n\n\n\n\n\u3000\u3000", "")
    new_chapers.append(single_text)
    #print(single_text)
chapters = new_chapers

In [24]:
# loop all the chapters and find the places in the text
list_of_df = []

for chapter in chapters:

    the_city_list = []
    # loop through all the chapters and interate through all the places 
    # and count the number of times the place name appears in the text

    for element in all_the_places:
        the_city_list.append(chapter.count(element))
    the_city_list
    d = {'city_name': all_the_places, 'occur_list':the_city_list}
    df_rlws_top_cities = pd.DataFrame(data=d)
    df_city_novel_filtered = df_rlws_top_cities.loc[df_rlws_top_cities["occur_list"]>0]
    df_city_novel_filtered = df_city_novel_filtered.sort_values(by=['occur_list'])
    df_city_novel = df_city_novel_filtered.loc[df_city_novel_filtered["occur_list"]<150]
    list_of_df.append(df_city_novel)

In [34]:

merged_dfs = []
# loop though list_of_df and merge with df_total to get the coordinates
for df in list_of_df:
    df_city_novel = df
    df_city_novel = df_city_novel.merge(df_total, left_on='city_name', right_on='NAME_SIM')
    df_city_novel = df_city_novel.drop_duplicates(subset=['city_name'])
    merged_dfs.append(df_city_novel)


# save the dataframes to csv files
for i in range(len(merged_dfs)):
    merged_dfs[i].to_csv('../geo_tags/'+str(i)+'.csv', index=False)
    
